##### Seung-Hwan Oh, Seoul, Korea
##### Modified by Seung-Hwan Oh 2023.05.21
##### Do not copy without permission

In [79]:
import numpy as np
import cv2

data = {"info":{"description":"my-project-name"},"images":[{"id":1,"width":640,"height":960,"file_name":"286514.jpg"}],"annotations":[{"id":0,"iscrowd":0,"image_id":1,"category_id":1,"segmentation":[[278.97783685583477,69.50596252129473,292.06131215396084,93.21976149914823,267.52979596997443,108.75638841567294,246.26914861051958,123.47529812606477,238.90969375532367,133.28790459965933,247.90458302278535,158.63713798977858,266.7120787638416,172.53833049403752,284.70185729876493,177.4446337308348,283.88414009263204,165.17887563884162,296.9676153907581,165.17887563884162,337.0357584912692,192.1635434412266,364.0204262936542,165.17887563884162,373.83303276724877,178.26235093696766,391.0050940960392,183.98637137989783,414.71889307389273,168.44974446337312,427.8023683720188,155.36626916524705,431.8909544026832,142.282793867121,426.166933959753,142.282793867121,404.90628660029813,121.02214650766612,390.1873768899063,106.3032367972743,411.4480242493612,83.40715502555368,428.62008557815165,94.85519591141399,486.67800721358606,128.38160136286203,488.31344162585185,153.7308347529813,433.5263888149489,256.76320272572406,422.07834792908864,416.21805792163553,361.56727467525553,433.390119250426,313.32195951341566,432.57240204429314,207.01872271614138,407.22316865417383,223.37306683879896,261.6695059625214,175.94546888309196,125.11073253833052,185.75807535668653,107.12095400340718,205.38328830387562,93.21976149914823,223.37306683879896,88.31345826235096,254.44632067184838,73.59454855195914]],"bbox":[175.94546888309196,69.50596252129473,312.3679727427599,363.88415672913123],"area":67598.66151627107}],"categories":[{"id":1,"name":"top"},{"id":2,"name":"outer"},{"id":3,"name":"bottom"},{"id":4,"name":"shoe"},{"id":5,"name":"bag"},{"id":6,"name":"hat"}]}

segmentation_points = data["annotations"][0]["segmentation"][0]
bbox = data["annotations"][0]["bbox"]

segmentation_array = np.array(segmentation_points).reshape((-1, 2)).astype(np.int32)

image_width = data["images"][0]["width"]
image_height = data["images"][0]["height"]
canvas = np.zeros((image_height, image_width), dtype=np.uint8)

cv2.fillPoly(canvas, [segmentation_array], color=(255, 255, 255))

output_file_name = "segmentation_image.jpg"
cv2.imwrite(output_file_name, canvas)

from PIL import Image

def binarize_image(image):
    # Convert the image to grayscale
    grayscale_image = image.convert('L')

    # Binarize the grayscale image using a threshold value
    threshold = 128  # Adjust the threshold value as needed
    binary_image = grayscale_image.point(lambda x: 0 if x < threshold else 255, '1')

    return binary_image

def run_length_encode(mask):
    rle = []
    count = 0
    is_running = False

    # Iterate over each pixel in the mask
    for pixel in mask.getdata():
        if pixel == 255:  # Assuming 255 represents the object in the mask
            if not is_running:
                start = count + 1  # Start of a new run
                is_running = True
        else:
            if is_running:
                length = count - start + 1  # Length of the run
                rle.append((start, length))
                is_running = False
        count += 1

    # Handle the case when the mask ends with a run
    if is_running:
        length = count - start + 1
        rle.append((start, length))

    return rle

def run_length_decode(rle, mask_shape):
    mask = Image.new('L', mask_shape)
    for start, length in rle:
        for i in range(length):
            x = (start + i) % mask_shape[0]
            y = (start + i) // mask_shape[0]
            mask.putpixel((x, y), 255)  # Set pixel value to 255
    return mask

# Load the original image
image = Image.open('segmentation_image.jpg')

# Binarize the image
binary_image = binarize_image(image)

# Run-length encode the binary mask
rle_encoded = run_length_encode(binary_image)

rle_encoded = ' '.join([' '.join([str(i[0]),str(i[1])]) for i in rle_encoded])
rle_encoded

In [81]:
from PIL import Image

def binarize_image(image):
    # Convert the image to grayscale
    grayscale_image = image.convert('L')

    # Binarize the grayscale image using a threshold value
    threshold = 128  # Adjust the threshold value as needed
    binary_image = grayscale_image.point(lambda x: 0 if x < threshold else 255, '1')

    return binary_image

def run_length_encode(mask):
    rle = []
    count = 0
    is_running = False

    # Iterate over each pixel in the mask
    for pixel in mask.getdata():
        if pixel == 255:  # Assuming 255 represents the object in the mask
            if not is_running:
                start = count + 1  # Start of a new run
                is_running = True
        else:
            if is_running:
                length = count - start + 1  # Length of the run
                rle.append((start, length))
                is_running = False
        count += 1

    # Handle the case when the mask ends with a run
    if is_running:
        length = count - start + 1
        rle.append((start, length))

    return rle

def run_length_decode(rle, mask_shape):
    mask = Image.new('L', mask_shape)
    for start, length in rle:
        for i in range(length):
            x = (start + i) % mask_shape[0]
            y = (start + i) // mask_shape[0]
            mask.putpixel((x, y), 255)  # Set pixel value to 255
    return mask

# Load the original image
image = Image.open('segmentation_image.jpg')

# Binarize the image
binary_image = binarize_image(image)

# Run-length encode the binary mask
rle_encoded = run_length_encode(binary_image)

# Save the run-length encoded data to a file
with open('segmentation_mask_rle.txt', 'w') as file:
    for start, length in rle_encoded:
        file.write(f"{start},{length}\n")

# Create a new segmentation mask from the run-length encoded data
mask_shape = binary_image.size
decoded_mask = run_length_decode(rle_encoded, mask_shape)

# Save the decoded mask image
decoded_mask.save('decoded_mask22.png')

In [47]:
import json
import numpy as np
import cv2
from PIL import Image
import pandas as pd

with open("labels_my-project-name.json", "r") as st_json:
    data = json.load(st_json)


In [73]:
imageid = []
width = []
height = []
classid = []

rle = []

sdict = {}

for i in range(len(data['annotations'])):
    imageid.append(data['images'][i]['file_name'])
    width.append(data['images'][i]['width'])
    height.append( data['images'][i]['height'])
    classid.append(data["annotations"][i]['category_id'])


    segmentation_points = data["annotations"][i]["segmentation"][0]
    bbox = data["annotations"][i]["bbox"]
    segmentation_array = np.array(segmentation_points).reshape((-1, 2)).astype(np.int32)
    image_width = data["images"][i]["width"]
    image_height = data["images"][i]["height"]
    canvas = np.zeros((image_height, image_width), dtype=np.uint8)
    cv2.fillPoly(canvas, [segmentation_array], color=(255, 255, 255))
    output_file_name = "segmentation_image.jpg"
    cv2.imwrite(output_file_name, canvas)
    
    def binarize_image(image):
        # Convert the image to grayscale
        grayscale_image = image.convert('L')
        # Binarize the grayscale image using a threshold value
        threshold = 128  # Adjust the threshold value as needed
        binary_image = grayscale_image.point(lambda x: 0 if x < threshold else 255, '1')
        return binary_image
    def run_length_encode(mask):
        rle = []
        count = 0
        is_running = False
        # Iterate over each pixel in the mask
        for pixel in mask.getdata():
            if pixel == 255:  # Assuming 255 represents the object in the mask
                if not is_running:
                    start = count + 1  # Start of a new run
                    is_running = True
            else:
                if is_running:
                    length = count - start + 1  # Length of the run
                    rle.append((start, length))
                    is_running = False
            count += 1
        # Handle the case when the mask ends with a run
        if is_running:
            length = count - start + 1
            rle.append((start, length))
        return rle
    def run_length_decode(rle, mask_shape):
        mask = Image.new('L', mask_shape)
        for start, length in rle:
            for i in range(length):
                x = (start + i) % mask_shape[0]
                y = (start + i) // mask_shape[0]
                mask.putpixel((x, y), 255)  # Set pixel value to 255
        return mask
    # Load the original image
    image = Image.open('segmentation_image.jpg')
    # Binarize the image
    binary_image = binarize_image(image)
    # Run-length encode the binary mask
    rle_encoded = run_length_encode(binary_image)
    rle_encoded = ' '.join([' '.join([str(i[0]),str(i[1])]) for i in rle_encoded])
    rle.append(rle_encoded)

In [74]:
import pandas as pd

f = pd.DataFrame({'imageid':imageid,
                    'EncodedPixcels': rle,
                    'width':width, 
                    'height':height, 
                    'classid':classid})

f.to_csv("data.csv",index=False)